In [1]:
from transformers import pipeline

class IntentClassifier:
    def __init__(self, model_name="bert-base-uncased"):
        self.pipeline = pipeline("text-classification", model=model_name, return_all_scores=True)
        # Map model's generic labels to intent names
        self.label_to_intent = {
            "LABEL_0": "find_flights",
            "LABEL_1": "list_activities",
            "LABEL_2": "budget_suggestions"
        }

    def classify_intent(self, user_input):
        predictions = self.pipeline(user_input)
        print("Predictions:", predictions)  # Debugging output
        
        # Extract scores and map to intents
        intent_scores = {self.label_to_intent.get(pred["label"], "unknown"): pred["score"] for pred in predictions[0]}
        best_intent = max(intent_scores, key=intent_scores.get)
        
        return best_intent, intent_scores


c:\Users\roham\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

class ResponseGenerator:
    def __init__(self, model_name="microsoft/DialoGPT-medium"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

    def generate_response(self, user_input):
        inputs = self.tokenizer.encode(user_input, return_tensors="pt")
        reply_ids = self.model.generate(inputs, max_length=50, pad_token_id=self.tokenizer.eos_token_id)
        return self.tokenizer.decode(reply_ids[0], skip_special_tokens=True)


In [3]:
import requests
import pandas as pd

def generate_bearer_token():

    url = "https://test.api.amadeus.com/v1/security/oauth2/token"

    payload = 'client_id=&client_secret=&grant_type=client_credentials'
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.json())
    return response.json()


def fetch_flights(destination, dates):
    bearer_token = generate_bearer_token()
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=SYD&destinationLocationCode=BKK&departureDate=2024-12-24&adults=2&max=5"

    payload = {}
    headers = {
    'Authorization': 'Bearer '
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    #print(response.text)

    # # Example function to fetch flight data
    # api_url = f"https://api.skyscanner.net/...&destination={destination}&dates={dates}"
    # response = requests.get(api_url)
    return response.json()

def fetch_activities(destination):
    # Example function to fetch activities
    api_url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=activities+in+{destination}"
    response = requests.get(api_url)
    return response.json()

def generate_itinerary(user_input):
    # Parse user input
    destination = user_input.get('destination')
    dates = user_input.get('dates')
    preferences = user_input.get('preferences', [])
    
    # Fetch data
    flights = fetch_flights(destination, dates)
    activities = fetch_activities(destination)
    
    # Create itinerary (simplified)
    itinerary = {
        "destination": destination,
        "dates": dates,
        "flights": flights.get('results', []),
        "activities": [act['name'] for act in activities.get('results', [])]
    }
    return itinerary


In [ ]:
from flask import Flask, render_template, request, jsonify
app = Flask(__name__)

# Load AI models
intent_classifier = IntentClassifier()
response_generator = ResponseGenerator()

@app.route('/')
def index():
    return render_template('index.html')  # Serve the frontend UI

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message', '')
    
    # Classify intent
    intent, scores = intent_classifier.classify_intent(user_message)
    print(intent)
    # Handle intent
    if intent == "find_flights":
        destination = "New York"  # Extract destination from NLP or user input
        dates = "2024-12-25"  # Extract dates
        flights = fetch_flights(destination, dates)
        response = f"Here are some flights to {destination} on {dates}: {flights}"
        # pass
        # response = response_generator.generate_response(user_message)
    # elif intent == "list_activities":
    #     destination = "New York"  # Extract from NLP
    #     activities = fetch_activities(destination)
    #     response = f"Here are some activities in {destination}: {activities}"
    else:
        # Generate conversational response
        response = response_generator.generate_response(user_message)
        print("Check")

    return jsonify({'response': response})

if __name__ == '__main__':
   try:
        app.run(debug=True,use_reloader=False)
   except Exception as e:
        print(f"Error occurred: {e}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
c:\Users\roham\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Dec/2024 05:49:17] "POST /chat HTTP/1.1" 200 -


Predictions: [[{'label': 'LABEL_0', 'score': 0.4675496220588684}, {'label': 'LABEL_1', 'score': 0.5324503779411316}]]
list_activities


127.0.0.1 - - [24/Dec/2024 05:49:27] "POST /chat HTTP/1.1" 200 -


...

Predictions: [[{'label': 'LABEL_0', 'score': 0.44853541254997253}, {'label': 'LABEL_1', 'score': 0.5514646172523499}]]
list_activities


127.0.0.1 - - [24/Dec/2024 05:49:33] "POST /chat HTTP/1.1" 200 -


Predictions: [[{'label': 'LABEL_0', 'score': 0.44853541254997253}, {'label': 'LABEL_1', 'score': 0.5514646172523499}]]
list_activities


127.0.0.1 - - [24/Dec/2024 05:49:35] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2024 05:49:40] "POST /chat HTTP/1.1" 200 -


Predictions: [[{'label': 'LABEL_0', 'score': 0.47550514340400696}, {'label': 'LABEL_1', 'score': 0.5244948267936707}]]
list_activities
